In [ ]:
sys.path.append('/workspace/MT/t2t_data_generators/')

In [ ]:
from generator import ShadENRUOpusProblem

In [ ]:
shad_problem = ShadENRUOpusProblem()

In [ ]:
data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TRAIN
)

In [ ]:
has_input = True
mode=tf.estimator.ModeKeys.TRAIN

hparams = transformer.transformer_base_single_gpu()
hparams.data_dir =  '/workspace/MT/shad_nlp18_contextNMT/data_4prev//'
p_hparams = shad_problem.get_hparams(hparams)

## Initializing the model

In [ ]:
model = TransformerCache(hparams, mode, p_hparams)

In [ ]:
out_logits, _ = model(features)

In [ ]:
save_dict = {"/".join(["transformer"] + var.name[:-2].split('/')[1:]) : var for var in tf.global_variables()[3:]}

In [ ]:
saver = tf.train.Saver(save_dict)
ckpt = tf.train.get_checkpoint_state('/workspace/MT/train/transformer/')

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("...no checkpoint found...")

In [ ]:
features_true = {
    "inputs": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "targets": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "target_space_id": tf.constant(1, dtype=tf.int32)
}

In [ ]:
out_logits, _ = model(features_true)

logits = tf.squeeze(out_logits, axis=[2, 3])

loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=tf.reshape(out_logits, [-1, VOCAB_SIZE]),
    labels=tf.reshape(features_true["targets"], [-1]))
loss = tf.reduce_mean(loss)

apply_grad = tf.train.AdamOptimizer(0.001).minimize(loss, var_list=tf.global_variables()[:2])

In [ ]:
#initialize adam's stuff
sess.run(tf.initialize_variables(tf.global_variables()[-6:]))

In [ ]:
data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TRAIN
)

In [ ]:
new_saver = tf.train.Saver(tf.global_variables()[:2])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

## Fine-tuning the mixing matrices for cache vector and transformer embedding

In [ ]:
loss_data = []
mean_loss = []
iteration = 0
glob_iteration = 0
epoch = 0

for it in data:
    
    inputs = np.array(it['inputs']).reshape((1, len(it['inputs']), 1, 1))
    targets = np.array(it['targets']).reshape((1, len(it['targets']), 1, 1))
    
    l, _ = sess.run([loss, apply_grad], feed_dict={
        features_true['inputs'] : inputs,
        features_true['targets'] : targets
    })
    
    loss_data.append(l)
    
    iteration += 1
    glob_iteration += 1
    
    if iteration == 5:
        model.sentence_cache.Flush()
        iteration = 0
    
    if len(loss_data) == 1000:
        clear_output(True)
        mean_loss.append(np.mean(loss_data))
        loss_data = []
        print("Epoch {}, Iteration {}, Mean loss {}".format(epoch, glob_iteration, mean_loss[-1]))
        
    if glob_iteration == 750000:
        epoch += 1
        glob_iteration = 0
        new_saver.save(sess, '/workspace/MT/train_cache/transformer_cache')